<a href="https://colab.research.google.com/github/Riccardo-Venturi/Tesi_Script_Colab/blob/main/CucituraScan_Separate_Orizzontale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -r /content/drive/MyDrive/progetto_fori_yolo/freeze_cpu_YOLOSCRIPTfori_2025-06-26.txt --quiet

In [ ]:
# Ubuntu / Colab
!sudo apt-get update
!sudo apt-get -y install exiftool jhead imagemagick


In [ ]:
#orientazione rotta delle immagini mogrify -auto-orient *.jpg
!mogrify -auto-orient *.jpg # per settare la rotazione in automatica su tutte le immagini in base al tag

In [ ]:
#@title INFERENZA IMMAGINI sia DNT/Optic; settaggio completo; usare per inferenze future sulle SCAN
#
#pass all scans!!
import cv2
import os
import glob
import pandas as pd
from pathlib import Path
from ultralytics import YOLO
IMG_DIR  = "/content/"#"/content/scans/Radio"#vecchie per scan ottiche#"/content/scan_ruotate_da_inferire_ingressi"#"/content/drive/MyDrive/Scan_Orientate"#"/content/scans"
CSV_DIR  = "NUOVE_CSV_RADIO"#"/content/scansCSV_oriented"
os.makedirs(CSV_DIR, exist_ok=True)
MODEL = YOLO("/content/drive/MyDrive/Pesi/best.pt") #pezzo precedente, adesso richiamo dal cloud diretto #/content/PESI/best.pt")#"/content/weights/best.pt")   # <- path you used

for img_path in sorted(glob.glob(f"{IMG_DIR}/*.jpg")):
    H,W = cv2.imread(img_path).shape[:2]
    res = MODEL.predict(img_path, imgsz=(H,W), conf=0.79,
                        iou=0.80,
                        )[0]#verbose=False)[0]
    if not len(res.boxes):
        print("⚠️  no boxes in", img_path);  continue

    df = pd.DataFrame(res.boxes.xyxy.cpu().numpy().astype(int),
                      columns=["x1","y1","x2","y2"])
    df.insert(0, "conf", res.boxes.conf.cpu().numpy())
    stem = Path(img_path).stem
    df.to_csv(f"{CSV_DIR}/{stem}.csv", index=False)
    print("•", stem, "→ CSV ok")
print("🎉  all scans processed →", CSV_DIR)


In [ ]:
# === ORDINA BOUNDING BOX IN SERPENTINA PER COLONNE (↓↑ a colonne alternate) ===
import os, numpy as np, pandas as pd
from sklearn.cluster import KMeans
from sklearn.linear_model import RANSACRegressor

def order_by_columns_boustro(df: pd.DataFrame,
                             n_cols: int = 22,
                             start_bottom_up: bool = True,
                             deskew_cols: bool = True,
                             expected_rows: int | None = 6):
    """
    Ordina i box (x1,y1,x2,y2) boustrafedico per COLONNE:
      - colonne ordinate da sinistra → destra
      - dentro ogni colonna: alterna BASSO→ALTO, poi ALTO→BASSO, ...
    start_bottom_up=True  -> 1ª colonna: BASSO→ALTO (coerente con 'dal basso in sinistra sale')
    Nota: in immagini tipo OpenCV, y cresce verso il basso.
    """

    df = df.reset_index(drop=True).copy()
    # Centri bbox
    cx = (df['x1'] + df['x2']) * 0.5
    cy = (df['y1'] + df['y2']) * 0.5

    # (opzionale) deskew delle colonne: x ≈ a*y + b  ->  x_corr = x - (a*y+b)
    if deskew_cols:
        ransac = RANSACRegressor(random_state=0)
        ransac.fit(cy.values.reshape(-1,1), cx.values)
        a = float(getattr(ransac.estimator_, "coef_", [0])[0])
        b = float(getattr(ransac.estimator_, "intercept_", 0))
        x_corr = cx - (a*cy + b)
    else:
        x_corr = cx.copy()

    # Clusterizza in N colonne sui valori x_corr
    km = KMeans(n_clusters=n_cols, n_init=25, random_state=0).fit(x_corr.values.reshape(-1,1))
    labels = km.labels_

    # Ordine delle colonne: sinistra -> destra (uso il centroide medio su x_corr)
    col_means = np.array([x_corr[labels==c].mean() for c in range(n_cols)])
    col_order = np.argsort(col_means)

    # Ordina boustrafedico dentro ogni colonna
    ordered_idx = []
    per_col_counts = []
    for j, c in enumerate(col_order):
        idxs = np.where(labels==c)[0]
        per_col_counts.append(len(idxs))

        # y cresce verso il basso: "BASSO→ALTO" = ordinare per y DECRESCENTE
        y_vals = cy.iloc[idxs].values
        order = np.argsort(y_vals)          # ALTO→BASSO (y piccolo → y grande)
        if start_bottom_up:                  # alterna
            if j % 2 == 0:
                order = order[::-1]          # 1ª colonna: BASSO→ALTO
        else:
            if j % 2 == 1:
                order = order[::-1]

        ordered_idx.extend(list(idxs[order]))

    out = df.iloc[ordered_idx].copy()
    out.insert(0, "hole_idx", np.arange(1, len(out)+1, dtype=int))

    # log utile
    if expected_rows is not None:
        bad = [k for k,c in enumerate(per_col_counts) if abs(c - expected_rows) > 1]
        print(f"Colonne: {n_cols} | media fori/colonna = {np.mean(per_col_counts):.2f}")
        if bad:
            print("⚠️  Colonne con conteggio anomalo (idx nella sequenza sinistra→destra):", bad)
    return out

# ====== CONFIG ======
IN_CSV  = "/content/NUOVE_CSV_RADIO/scan4_cucita_senza_linea.csv"   # <--- il tuo CSV
OUT_DIR = "/content/ordered_csv_fixed"
os.makedirs(OUT_DIR, exist_ok=True)
OUT_CSV = os.path.join(OUT_DIR, "scan4_cucita_senza_linea_boustro_cols.csv")

# ====== RUN ======
df_in  = pd.read_csv(IN_CSV)
df_out = order_by_columns_boustro(df_in, n_cols=22, start_bottom_up=True, deskew_cols=True, expected_rows=6)
df_out.to_csv(OUT_CSV, index=False)

print("✅ Salvato CSV ordinato →", OUT_CSV)
print("Prime 12 righe ordinate:")
display(df_out.head(12))
print("N fori totali:", len(df_out))


In [ ]:
import cv2, os, zipfile, re
import pandas as pd
from pathlib import Path

# ====== CONFIG ======
CSV_INPUT  = "/content/ordered_csv_fixed/scan4_cucita_senza_linea_boustro_cols.csv"  # file singolo O directory
SCAN_DIR   = "/content"                         # dove stanno le immagini .jpg/.png
PATCH_DIR  = "/content/Radio_Patches_Output"
ZIP_PATH   = "/content/Radio_Patches_Output.zip"
HS         = 224                                # half-size: 170 -> crop 340x340 centrato sul foro
TARGET_SIZE= 512                                # output patch size

# ====== UTIL ======
def list_csvs(csv_input: str):
    p = Path(csv_input)
    if p.is_file():
        return [p]
    elif p.is_dir():
        return sorted(p.glob("*.csv"))
    else:
        raise FileNotFoundError(f"CSV_INPUT non esiste: {csv_input}")

def guess_image_path(csv_path: Path, scan_dir: str) -> Path:
    """
    Dato un CSV tipo 'scan4_cucita_senza_linea_boustro_cols.csv'
    prova a mappare a 'scan4_cucita_senza_linea.jpg' (o .png/.jpeg).
    """
    base = csv_path.stem
    # rimuovi suffissi noti introdotti dall'ordinamento
    base = re.sub(r'(_boustro(_cols)?|_ordered(_NEW)?|_ordered_NEW|_ordered|_fixed)$', '', base)
    # candidati comuni
    cand = [Path(scan_dir)/f"{base}{ext}" for ext in (".jpg",".png",".jpeg",".JPG",".PNG",".JPEG")]
    for c in cand:
        if c.exists():
            return c
    # fallback: cerca per prefisso (es. "scan4_cucita_senza_linea*")
    for p in sorted(Path(scan_dir).glob(f"{base}*.jpg")) + \
             sorted(Path(scan_dir).glob(f"{base}*.png")) + \
             sorted(Path(scan_dir).glob(f"{base}*.jpeg")):
        return p
    raise FileNotFoundError(f"Nessuna immagine trovata per base '{base}' in {scan_dir}")

def crop_square(img, cx, cy, hs):
    H, W = img.shape[:2]
    x1, y1 = int(cx - hs), int(cy - hs)
    x2, y2 = int(cx + hs), int(cy + hs)
    # clamp ai bordi
    x1 = max(0, x1); y1 = max(0, y1)
    x2 = min(W, x2); y2 = min(H, y2)
    return img[y1:y2, x1:x2]

# ====== PREP ======
os.makedirs(PATCH_DIR, exist_ok=True)
if os.path.exists(ZIP_PATH):
    os.remove(ZIP_PATH)

csv_files = list_csvs(CSV_INPUT)
if not csv_files:
    raise SystemExit(f"Nessun CSV trovato in: {CSV_INPUT}")

global_idx = 1
print(f"🚀 Cropping {len(csv_files)} CSV…")

with zipfile.ZipFile(ZIP_PATH, "w", compression=zipfile.ZIP_DEFLATED) as zf:
    for csv_path in csv_files:
        print(f"\n📄 CSV: {csv_path.name}")
        try:
            img_path = guess_image_path(csv_path, SCAN_DIR)
        except FileNotFoundError as e:
            print(f"⚠️  {e} — salto.")
            continue

        img = cv2.imread(str(img_path), cv2.IMREAD_COLOR)
        if img is None:
            print(f"⚠️  Impossibile leggere immagine: {img_path} — salto.")
            continue

        H_img, W_img = img.shape[:2]
        print(f"🖼️  Immagine: {img_path.name} ({W_img}x{H_img})")

        df = pd.read_csv(csv_path)
        if "hole_idx" in df.columns:
            df = df.sort_values("hole_idx").reset_index(drop=True)
        else:
            # fallback: ordina per y poi x se non c'è hole_idx
            df = df.sort_values(by=["y1","x1"]).reset_index(drop=True)

        stem = csv_path.stem
        outdir = Path(PATCH_DIR)/stem
        outdir.mkdir(parents=True, exist_ok=True)

        for _, r in df.iterrows():
            cx = (float(r.x1) + float(r.x2)) * 0.5
            cy = (float(r.y1) + float(r.y2)) * 0.5
            patch = crop_square(img, cx, cy, HS)

            # se il crop è più piccolo (vicino ai bordi), ridimensioniamo comuqnue al target
            patch = cv2.resize(patch, (TARGET_SIZE, TARGET_SIZE), interpolation=cv2.INTER_LANCZOS4)

            hid = int(r.hole_idx) if "hole_idx" in df.columns else global_idx
            fname = f"H{global_idx:03d}_h{hid:03d}_{stem}.jpg"
            fpath = outdir/fname
            cv2.imwrite(str(fpath), patch)

            zf.write(str(fpath), arcname=f"{stem}/{fname}")
            global_idx += 1

        print(f"✔  Create {len(df)} patch per '{stem}'")

print(f"\n🎉 Fatto! Patch totali: {global_idx-1} → {PATCH_DIR}")
print(f"   ZIP: {ZIP_PATH}")


In [ ]:
import cv2
import pandas as pd
from pathlib import Path

def crop_from_csv(csv_path, img_path, out_dir, hs):
    """
    Taglia patch quadrate centrata sul foro da CSV YOLO.
    hs = half-size in pixel (es. 256 -> patch 512x512)
    Nessun resize o padding.
    """
    Path(out_dir).mkdir(parents=True, exist_ok=True)

    img = cv2.imread(str(img_path))
    if img is None:
        raise FileNotFoundError(f"Immagine non trovata: {img_path}")
    H_img, W_img = img.shape[:2]

    df = pd.read_csv(csv_path)
    if "hole_idx" in df.columns:
        df = df.sort_values("hole_idx").reset_index(drop=True)
    else:
        df = df.sort_values(by=["y1", "x1"]).reset_index(drop=True)

    for idx, r in df.iterrows():
        cx = (float(r.x1) + float(r.x2)) / 2
        cy = (float(r.y1) + float(r.y2)) / 2

        x1, y1 = int(cx - hs), int(cy - hs)
        x2, y2 = int(cx + hs), int(cy + hs)

        # clamp ai bordi
        x1 = max(0, x1)
        y1 = max(0, y1)
        x2 = min(W_img, x2)
        y2 = min(H_img, y2)

        patch = img[y1:y2, x1:x2]
        out_name = f"patch_{idx+1:03d}.jpg"
        cv2.imwrite(str(Path(out_dir) / out_name), patch)

    print(f"✔ Creati {len(df)} crop in: {out_dir}")

# ===== ESEMPIO USO =====
csv_file = "/content/ordered_csv_fixed/scan4_cucita_senza_linea_boustro_cols.csv"
img_file = "/content/scan4_cucita_senza_linea.jpg"
output_dir = "/content/crop_out"
HS = 256  # half-size => crop 512x512

crop_from_csv(csv_file, img_file, output_dir, HS)


In [ ]:
#@title metadata are fucked up cause the professor rotated the image wrong and i manually rotated it and open cv reads metadata not tags of rotation... one day wasted...


In [ ]:
import cv2
from google.colab.patches import cv2_imshow

PATH_SUP = "/content/T_0_90_5 (Lato SUP).jpg"

# leggi ignorando eventuali tag EXIF di rotazione
img_sup1 = cv2.imread(PATH_SUP, cv2.IMREAD_COLOR )#| cv2.IMREAD_IGNORE_ORIENTATION)

if img_sup1 is None:
    raise FileNotFoundError(PATH_SUP)

print("Shape:", img_sup1.shape)   # (H, W, 3)   ← controlla che H ≈ 4300, W ≈ 9100

cv2_imshow(img_sup1)              # mostra l’intera immagine


#La sintassi generale è img[y_start:y_end, x_start:x_end].

Negativi: img[-N:, :] prende gli ultimi N pixel in altezza.

Intervallo assoluto: img[3000:4205, :] prende le righe da 3000 (inclusa) a 4205 (esclusa).

```Esempi pratici:
## strip = img[3000:4205, :].copy() righe pixels 3k a 4k

Perché [:-N, :] non prende “la fascia da 3000 a 4205”:

Negativi nelle slice

-N è un indice relativo “a partire dalla fine”.

:-N → «dalla riga 0 fino a (ma non inclusa) la riga len(img)-N».

Se N = 3205 e l’immagine è alta 6200 px, stai tenendo le righe 0 … 2994 (circa), non 3000 … 4205.

--------------------------------------------------------------------------------

Coordinate immagine

L’origine (0, 0) è in alto a sinistra.

y aumenta verso il basso, x aumenta verso destra.

Non c’è «X / Y invertiti»; è soltanto la convenzione grafica invece della convenzione cartesiana “matematica”.```

In [ ]:
H = img_sup1.shape[0]   # altezza
W = img_sup1.shape[1]   # larghezza
C = img_sup1.shape[2]   # canali (3 per BGR)

N = 3205                       # altezza della fascia in pixel
#Prendiamo l'ultimo 25 % di pixels verticali per fare la cucitura
strip = img_sup1[:int(0.25*H), :].copy() # tutti i canali, tutte le colonne

print("Strip shape:", strip.shape)  # (1000, 9100, 3) circa
cv2_imshow(strip)                   # quick-check


In [ ]:
#@title cucire strip su immagine inferiore
import cv2, numpy as np
from google.colab.patches import cv2_imshow   # serve solo in Colab

SUP = strip
INF = "/content/T_0_90_5 (Lato INF).jpg"


In [ ]:
import cv2, numpy as np
from google.colab.patches import cv2_imshow     # mostriamo in Colab

# 1. leggi l’immagine inferiore così com’è (niente rotazioni automatiche)
buf_inf = np.fromfile("/content/T_0_90_5 (Lato INF).jpg", np.uint8)
inf     = cv2.imdecode(buf_inf, cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)

# 2. strip ESISTE già (25 %) ─ se non l’hai ancora creato, fallo in 1 riga
#    strip = sup[:int(sup.shape[0]*0.25), :].copy()

# 3. padding orizzontale per pareggiare le larghezze (simmetrico, nero)
W = max(strip.shape[1], inf.shape[1])
pad = lambda im: cv2.copyMakeBorder(
        im, 0, 0, (W-im.shape[1])//2, (W-im.shape[1])//2,
        cv2.BORDER_CONSTANT, value=(0,0,0) )

strip = pad(strip)
inf   = pad(inf)

# 4. concatena verticalmente: strip SOPRA, inf SOTTO  (niente resize)
stitched = cv2.vconcat([strip, inf])

cv2_imshow(stitched)                             # anteprima
cv2.imwrite("/content/scan5_cucita.jpg", stitched)
print("FINALE:", stitched.shape)


In [ ]:
import cv2, numpy as np, os
from google.colab.patches import cv2_imshow   # se vuoi una preview ogni tanto

# --- percorsi -----------------------------------------------------------------
SUP = "/content/T_0_90_5 (Lato SUP).jpg"
INF = "/content/T_0_90_5 (Lato INF).jpg"
OUT_DIR = "/content/scan5_shifts"   # cartella di destinazione

os.makedirs(OUT_DIR, exist_ok=True)

# --- funzioni utili -----------------------------------------------------------
def read_raw(p):
    buf = np.fromfile(p, np.uint8)
    return cv2.imdecode(buf, cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)

def center_pad(img, target_w):
    pad = target_w - img.shape[1]
    if pad <= 0: return img
    L = pad // 2
    R = pad - L
    return cv2.copyMakeBorder(img, 0, 0, L, R,
                              cv2.BORDER_CONSTANT, value=(0,0,0))

def hshift(img, dx, color=(0,0,0)):
    h, w = img.shape[:2]
    if dx == 0: return img
    if dx > 0:   # → destra
        return np.hstack([np.full((h, dx, 3), color, img.dtype), img[:, :w-dx]])
    else:        # ← sinistra
        dx = -dx
        return np.hstack([img[:, dx:], np.full((h, dx, 3), color, img.dtype)])

# --- lettura immagini ---------------------------------------------------------
sup = read_raw(SUP)
inf = read_raw(INF)

# --- estrai la fascia alta (25 %) --------------------------------------------
strip_height = int(sup.shape[0] * 0.25)
strip_orig   = sup[:strip_height, :].copy()      # copia di sicurezza

# --- prepara larghezze uguali -------------------------------------------------
W_target = max(strip_orig.shape[1], inf.shape[1])
strip_orig = center_pad(strip_orig, W_target)
inf        = center_pad(inf,        W_target)

# -----------------------------------------------------------------------------
# loop sugli shift: da 300 a 990 (step 10)   (↔  sposta a sinistra = negativo)
# -----------------------------------------------------------------------------
for dx in range(300, 1000, 10):
    strip = hshift(strip_orig, -dx)                 # shift verso sinistra di dx px
    stitched = cv2.vconcat([strip, inf])            # nessun resize

    out_name = f"scan5_shift_{dx:04d}.jpg"
    cv2.imwrite(os.path.join(OUT_DIR, out_name), stitched)

print("✔  Salvate", len(os.listdir(OUT_DIR)), "immagini in", OUT_DIR)


In [ ]:
#@title INFERENZA IMMAGINI sia DNT/Optic; settaggio completo; usare per inferenze future sulle SCAN
#
#pass all scans!!
import cv2
import os
import glob
import pandas as pd
from pathlib import Path
from ultralytics import YOLO
#IMG_DIR  = "/content/scan5_shifts/scan5_shift_0610.jpg"#"/content/scans/Radio"#vecchie per scan ottiche#"/content/scan_ruotate_da_inferire_ingressi"#"/content/drive/MyDrive/Scan_Orientate"#"/content/scans"
CSV_DIR  = "NUOVE_CSV_RADIO"#"/content/scansCSV_oriented"
os.makedirs(CSV_DIR, exist_ok=True)
MODEL = YOLO("/content/drive/MyDrive/Pesi/best.pt") #pezzo precedente, adesso richiamo dal cloud diretto #/content/PESI/best.pt")#"/content/weights/best.pt")   # <- path you used
img_path = "/content/scan5_shifts/scan5_shift_0610.jpg"
#for img_path in sorted(glob.glob(f"{IMG_DIR}/*.jpg")):
H,W = cv2.imread(img_path).shape[:2]
res = MODEL.predict(img_path, imgsz=(H,W), conf=0.79,
                    iou=0.80,
                    )[0]#verbose=False)[0]
if not len(res.boxes):
    print("⚠️  no boxes in", img_path);
df = pd.DataFrame(res.boxes.xyxy.cpu().numpy().astype(int),
                  columns=["x1","y1","x2","y2"])
df.insert(0, "conf", res.boxes.conf.cpu().numpy())
stem = Path(img_path).stem
df.to_csv(f"{CSV_DIR}/{stem}.csv", index=False)
print("•", stem, "→ CSV ok")
print("🎉  all scans processed →", CSV_DIR)


In [ ]:
# === ORDINA BOUNDING BOX IN SERPENTINA PER COLONNE (↓↑ a colonne alternate) ===
import os, numpy as np, pandas as pd
from sklearn.cluster import KMeans
from sklearn.linear_model import RANSACRegressor

def order_by_columns_boustro(df: pd.DataFrame,
                             n_cols: int = 14,
                             start_bottom_up: bool = True,
                             deskew_cols: bool = True,
                             expected_rows: int | None = 6):
    """
    Ordina i box (x1,y1,x2,y2) boustrafedico per COLONNE:
      - colonne ordinate da sinistra → destra
      - dentro ogni colonna: alterna BASSO→ALTO, poi ALTO→BASSO, ...
    start_bottom_up=True  -> 1ª colonna: BASSO→ALTO (coerente con 'dal basso in sinistra sale')
    Nota: in immagini tipo OpenCV, y cresce verso il basso.
    """

    df = df.reset_index(drop=True).copy()
    # Centri bbox
    cx = (df['x1'] + df['x2']) * 0.5
    cy = (df['y1'] + df['y2']) * 0.5

    # (opzionale) deskew delle colonne: x ≈ a*y + b  ->  x_corr = x - (a*y+b)
    if deskew_cols:
        ransac = RANSACRegressor(random_state=0)
        ransac.fit(cy.values.reshape(-1,1), cx.values)
        a = float(getattr(ransac.estimator_, "coef_", [0])[0])
        b = float(getattr(ransac.estimator_, "intercept_", 0))
        x_corr = cx - (a*cy + b)
    else:
        x_corr = cx.copy()

    # Clusterizza in N colonne sui valori x_corr
    km = KMeans(n_clusters=n_cols, n_init=25, random_state=0).fit(x_corr.values.reshape(-1,1))
    labels = km.labels_

    # Ordine delle colonne: sinistra -> destra (uso il centroide medio su x_corr)
    col_means = np.array([x_corr[labels==c].mean() for c in range(n_cols)])
    col_order = np.argsort(col_means)

    # Ordina boustrafedico dentro ogni colonna
    ordered_idx = []
    per_col_counts = []
    for j, c in enumerate(col_order):
        idxs = np.where(labels==c)[0]
        per_col_counts.append(len(idxs))

        # y cresce verso il basso: "BASSO→ALTO" = ordinare per y DECRESCENTE
        y_vals = cy.iloc[idxs].values
        order = np.argsort(y_vals)          # ALTO→BASSO (y piccolo → y grande)
        if start_bottom_up:                  # alterna
            if j % 2 == 0:
                order = order[::-1]          # 1ª colonna: BASSO→ALTO
        else:
            if j % 2 == 1:
                order = order[::-1]

        ordered_idx.extend(list(idxs[order]))

    out = df.iloc[ordered_idx].copy()
    out.insert(0, "hole_idx", np.arange(1, len(out)+1, dtype=int))

    # log utile
    if expected_rows is not None:
        bad = [k for k,c in enumerate(per_col_counts) if abs(c - expected_rows) > 1]
        print(f"Colonne: {n_cols} | media fori/colonna = {np.mean(per_col_counts):.2f}")
        if bad:
            print("⚠️  Colonne con conteggio anomalo (idx nella sequenza sinistra→destra):", bad)
    return out

# ====== CONFIG ======
IN_CSV  = "/content/NUOVE_CSV_RADIO/scan5_shift_0610.csv"   # <--- il tuo CSV
OUT_DIR = "/content/ordered_csv_fixed"
os.makedirs(OUT_DIR, exist_ok=True)
OUT_CSV = os.path.join(OUT_DIR, "scan5_cucita_senza_linea_boustro_cols_Finale.csv")

# ====== RUN ======
df_in  = pd.read_csv(IN_CSV)
df_out = order_by_columns_boustro(df_in, n_cols=14, start_bottom_up=True, deskew_cols=True, expected_rows=6)
df_out.to_csv(OUT_CSV, index=False)

print("✅ Salvato CSV ordinato →", OUT_CSV)
print("Prime 12 righe ordinate:")
display(df_out.head(12))
print("N fori totali:", len(df_out))
